 <a name='ind'/>

# <ins>Indice</ins>:

- ## [Extracción](#ext)
- ## [Transformación](#trans)
- ## [Carga](#load)

In [1]:
import requests as rq
import pandas as pd
import pyarrow
from tqdm import tqdm
import time
import pymongo
from Spotify_id import *

En este notebook procederemos a obtener la información de Spotify para cada uno de los artistas de todos los festivales que hemos obtenido en el notebook Songkick.ipynb. Para ello nos ayudaremos de la API, y de la [documentación](https://developer.spotify.com/documentation/web-api) disponible en internet para el uso de la misma

<a name='ext'/>

###### ⬆️ [Indice](#ind)

### Extracción
***

In [2]:
# Obtener el TOKEN

AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

<Response [200]>

In [3]:
# Cargamos los grupos

grupos = pd.read_parquet('../data/grupos_spotify_clean.parquet')
grupos

,Grupos,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,Del Water Gap,{'spotify': 'https://open.spotify.com/artist/0...,"{'href': None, 'total': 122777}","[deep new americana, indie pop, pov: indie]",https://api.spotify.com/v1/artists/0xPoVNPnxII...,0xPoVNPnxIIUS1vrxAYV00,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Del Water Gap,58.0,artist,spotify:artist:0xPoVNPnxIIUS1vrxAYV00
1,León Benavente,{'spotify': 'https://open.spotify.com/artist/5...,"{'href': None, 'total': 74409}",[spanish indie pop],https://api.spotify.com/v1/artists/5wuBA4TiRM0...,5wuBA4TiRM0CNjSXjsutn6,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",León Benavente,42.0,artist,spotify:artist:5wuBA4TiRM0CNjSXjsutn6
2,ATB,{'spotify': 'https://open.spotify.com/artist/7...,"{'href': None, 'total': 354818}","[german dance, german techno, german trance, t...",https://api.spotify.com/v1/artists/7jZM5w05mGh...,7jZM5w05mGhw6wTB1okhD9,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",ATB,64.0,artist,spotify:artist:7jZM5w05mGhw6wTB1okhD9
3,Tyler Joe Miller,{'spotify': 'https://open.spotify.com/artist/1...,"{'href': None, 'total': 4670}",[canadian contemporary country],https://api.spotify.com/v1/artists/1MmpCgmJymS...,1MmpCgmJymS8Etwm9RxuxM,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Tyler Joe Miller,41.0,artist,spotify:artist:1MmpCgmJymS8Etwm9RxuxM
4,Loverground,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 4151}",[],https://api.spotify.com/v1/artists/3SvoerawAn5...,3SvoerawAn5RAZ2N9osc3z,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Loverground,38.0,artist,spotify:artist:3SvoerawAn5RAZ2N9osc3z
...,...,...,...,...,...,...,...,...,...,...,...
4610,Cari Cari,{'spotify': 'https://open.spotify.com/artist/0...,"{'href': None, 'total': 48689}",[austrian pop],https://api.spotify.com/v1/artists/0ey6crPk5fX...,0ey6crPk5fXQWkq21iPTMV,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Cari Cari,48.0,artist,spotify:artist:0ey6crPk5fXQWkq21iPTMV
4611,Iggy Pop,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 1475500}","[alternative rock, glam rock, permanent wave, ...",https://api.spotify.com/v1/artists/33EUXrFKGjp...,33EUXrFKGjpUSGacqEHhU4,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Iggy Pop,63.0,artist,spotify:artist:33EUXrFKGjpUSGacqEHhU4
4612,Great Northern,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 5994}","[deep acoustic pop, la indie]",https://api.spotify.com/v1/artists/3vEgkhVW7YF...,3vEgkhVW7YFsvSUjzWQ6ji,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Great Northern,21.0,artist,spotify:artist:3vEgkhVW7YFsvSUjzWQ6ji
4613,Morcheeba,{'spotify': 'https://open.spotify.com/artist/6...,"{'href': None, 'total': 772891}","[downtempo, electronica, trip hop]",https://api.spotify.com/v1/artists/6bWxFw65IEJ...,6bWxFw65IEJzBYjx3SxUXd,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Morcheeba,60.0,artist,spotify:artist:6bWxFw65IEJzBYjx3SxUXd


Tras consultar la documentación, podremos crear una lista de direcciones url para cada grupo que nos permitan lanzar las consultas a la API correctamente

In [4]:
# Para la consulta debemos sustituir los espacios por el caracter '+'
consulta = [x.replace(' ','+') for x in grupos.Grupos]

# Construimos una lista con la url correspondiente para cada grupo
urls = [f'https://api.spotify.com/v1/search?q=artist%3A{artist}&type=artist&limit=50&offset=0' for artist in consulta]

# Creamos una lista de diccionarios con la informacion de cada grupo y con su url
artist_url = [{'artist':artist,'url':url} for artist,url in zip(grupos.Grupos,urls)]

Para automatizar las consultas creamos la función *search_spotify_info()*, que recibe como argumento un diccionario con la información del grupo y su url de consulta para la API. Si la consulta es correcta, devolverá un json con la información del artista

In [5]:
def search_spotify_info(art_url):
    time.sleep(0.20)     # Pausamos el tiempo entre consultas. Evitamos que nos baneen
       # probamos conexión
    try:
        response = rq.get(art_url['url'],headers=headers)
        rjson = response.json()['artists']['items']
        for index, item in enumerate(rjson):
            if item['name'].lower() == art_url['artist'].lower():
                return rjson[index]
        
        raise Exception    # Si no se han encontrado el grupo lanzamos excepción para devolver None
    except:
        return{'external_urls': None,
               'followers': None,
               'genres': None,
               'href': None,
               'id': None,
               'images' : None,
               'name' : None,
               'popularity' : None,
               'type' : None,
               'uri' : None}

In [6]:
# Este proceso no puede ser paralelizable, ya que demasiadas peticiones por segundo a la API, provoca que se nos 
# revoquen los permisos

lst_artist_info = [search_spotify_info(art_url) for art_url in tqdm(artist_url)]

100%|██████████████████████████████████████████████████████████████████████████████| 5174/5174 [34:33<00:00,  2.50it/s]


<a name='trans'/>

###### ⬆️ [Indice](#ind)

### Transformación
***

In [7]:
grupos = pd.concat([grupos,pd.DataFrame(lst_artist_info)],axis=1)
grupos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5174 entries, 0 to 5173
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Grupos         5174 non-null   object 
 1   external_urls  4615 non-null   object 
 2   followers      4615 non-null   object 
 3   genres         4615 non-null   object 
 4   href           4615 non-null   object 
 5   id             4615 non-null   object 
 6   images         4615 non-null   object 
 7   name           4615 non-null   object 
 8   popularity     4615 non-null   float64
 9   type           4615 non-null   object 
 10  uri            4615 non-null   object 
dtypes: float64(1), object(10)
memory usage: 444.8+ KB


In [19]:
grupos.tail()

,Grupos,external_urls,followers,genres,href,id,images,name,popularity,type,uri
5169,Cari Cari,{'spotify': 'https://open.spotify.com/artist/0...,"{'href': None, 'total': 48689}",[austrian pop],https://api.spotify.com/v1/artists/0ey6crPk5fX...,0ey6crPk5fXQWkq21iPTMV,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Cari Cari,48.0,artist,spotify:artist:0ey6crPk5fXQWkq21iPTMV
5170,Iggy Pop,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 1475500}","[alternative rock, glam rock, permanent wave, ...",https://api.spotify.com/v1/artists/33EUXrFKGjp...,33EUXrFKGjpUSGacqEHhU4,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Iggy Pop,63.0,artist,spotify:artist:33EUXrFKGjpUSGacqEHhU4
5171,Great Northern,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 5994}","[deep acoustic pop, la indie]",https://api.spotify.com/v1/artists/3vEgkhVW7YF...,3vEgkhVW7YFsvSUjzWQ6ji,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Great Northern,21.0,artist,spotify:artist:3vEgkhVW7YFsvSUjzWQ6ji
5172,Morcheeba,{'spotify': 'https://open.spotify.com/artist/6...,"{'href': None, 'total': 772891}","[downtempo, electronica, trip hop]",https://api.spotify.com/v1/artists/6bWxFw65IEJ...,6bWxFw65IEJzBYjx3SxUXd,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Morcheeba,60.0,artist,spotify:artist:6bWxFw65IEJzBYjx3SxUXd
5173,H2O,{'spotify': 'https://open.spotify.com/artist/4...,"{'href': None, 'total': 90036}","[hardcore, nyhc, punk, skate punk, straight edge]",https://api.spotify.com/v1/artists/4mQm8gcXNNy...,4mQm8gcXNNyzNQRqYKTKqo,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",H2O,39.0,artist,spotify:artist:4mQm8gcXNNyzNQRqYKTKqo


In [31]:
# Guardamos los datos antes de transformarlos por seguridad

grupos.to_parquet('../data/grupos_spotify_raw.parquet',index=False,engine='pyarrow')

Hay grupos de los cuales no se ha podido obtener la información de la API, los separaremos del dataframe y los guardaremos aparte para poder completar esta información más adelanta

In [18]:
grupos_nan = grupos[grupos.id.isna()].reset_index(drop=True)
grupos_nan

,Grupos,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,Morgan (ES),None,None,None,None,None,None,None,NaN,None,None
1,Marwa Belhaj Youssef,None,None,None,None,None,None,None,NaN,None,None
2,THUMPER (IRE),None,None,None,None,None,None,None,NaN,None,None
3,SIMONA (BCN),None,None,None,None,None,None,None,NaN,None,None
4,MAREIKE BAUTZ,None,None,None,None,None,None,None,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...
554,VXYX,None,None,None,None,None,None,None,NaN,None,None
555,Harleigh Colt (DJ Set) of Rainbow Kitten Surpr...,None,None,None,None,None,None,None,NaN,None,None
556,Jamz Supernova,None,None,None,None,None,None,None,NaN,None,None
557,STOR,None,None,None,None,None,None,None,NaN,None,None


In [29]:
grupos_nan.to_parquet('../data/grupos_nan_spotify_raw.parquet',index=False,engine='pyarrow')

In [30]:
grupos = grupos.dropna().reset_index(drop=True)
grupos.to_parquet('../data/grupos_spotify_clean.parquet',index=False,engine='pyarrow')

<a name='load'/>

###### ⬆️ [Indice](#ind)

### Carga
***

Procederemos a subir la información a la DB de Mongo creada en el notebook Songkick.ipynb

In [33]:
cursor = pymongo.MongoClient('mongodb://localhost:27017')

In [34]:
festivales = cursor.Festivales

In [39]:
artist_colec = festivales.artist_colec

In [40]:
artist_colec.insert_many(grupos.to_dict(orient='records'))

In [42]:
artist_nan_colec = festivales.artist_nan_colec

In [43]:
artist_nan_colec.insert_many(grupos_nan.to_dict(orient='records'))